# Sección 2. Entendimiento y Preparación de los datos

Esta sección tiene como propósito exponer el entendimiento y la preparación de los textos para posteriormente utilizarlos en la implementación de los modelos de aprendizaje. Para la manipulación de los datos se usarán librerias como **pandas** y **numpy**. Para la visualización se utilizarán **matplotlib**, **seaborn** y **WordCloud**.

## 1. Instalación e importación de librerias

### Instalación

In [1]:
#%pip install pandas numpy matplotlib seaborn scikit-learn nltk spacy wordcloud tqdm
#%pip install spacy-langdetect
#!python -m spacy download es_core_news_sm


In [2]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Procesamiento de texto
import re
import string
import nltk
import spacy
from nltk.corpus import stopwords
from spacy_langdetect import LanguageDetector
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Progreso en bucles
from tqdm import tqdm

# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Descargar stopwords de NLTK en español
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10,5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2. Perfilamiento y entendimiento de los datos

### Carga de datos

In [3]:
# Cargar datos
data=pd.read_csv('fake_news_spanish.csv', sep=';', encoding = "utf-8")

news_df = data.copy()

In [4]:
news_df.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


In [5]:
news_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57063 entries, 0 to 57062
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           57063 non-null  object
 1   Label        57063 non-null  int64 
 2   Titulo       57047 non-null  object
 3   Descripcion  57063 non-null  object
 4   Fecha        57063 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


## 3. Preparación de los datos

### 3.1. Limpieza de datos.

Este código implementa un proceso de preprocesamiento de texto para limpiar datos textuales antes de realizar análisis de texto o procesamiento de lenguaje natural.El preprocesamiento se aplica a dos columnas de un DataFrame (Titulo y Descripcion), generando nuevas columnas con los textos limpios (Titulo_Limpio y Descripcion_Limpia). Este paso es fundamental porque ayuda a reducir el ruido en los datos, mejora la precisión de los modelos y facilita la extracción de información relevante eliminando términos irrelevantes o redundantes.


In [7]:
import nltk
import unicodedata
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


# Funciones de preprocesamiento
def remove_non_ascii(words):
    """Remueve caracteres no ASCII de una lista de palabras"""
    return [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words if word]

def to_lowercase(words):
    """Convierte todas las palabras a minúsculas"""
    return [word.lower() for word in words if word]

def remove_punctuation(words):
    """Elimina signos de puntuación"""
    return [re.sub(r'[^\w\s]', '', word) for word in words if re.sub(r'[^\w\s]', '', word) != '']

def remove_numbers(words):
    """Elimina los números de la lista de palabras"""
    return [word for word in words if not word.isdigit()]

def remove_stopwords(words):
    """Elimina stopwords en español"""
    return [word for word in words if word not in stopwords_es]

def preprocessing(text):
    """Aplica todas las funciones de limpieza de texto a un string"""
    words = word_tokenize(text, language="spanish")  # Tokenizar con NLTK
    words = to_lowercase(words)
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return " ".join(words)  # Retorna el texto limpio como un string

# Aplicar limpieza en el DataFrame
news_df['Titulo_Limpio'] = news_df['Titulo'].astype(str).apply(preprocessing)
news_df['Descripcion_Limpia'] = news_df['Descripcion'].astype(str).apply(preprocessing)

# Comparación antes y después
print("🔹 Comparación antes y después de la limpieza:")
print(news_df[['Titulo', 'Titulo_Limpio']].head())
print(news_df[['Descripcion', 'Descripcion_Limpia']].head())


🔹 Comparación antes y después de la limpieza:
                                              Titulo  \
0  'The Guardian' va con Sánchez: 'Europa necesit...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El 'Ahora o nunca' de Joan Fuster sobre el est...   
3  Iglesias alienta a Yolanda Díaz, ERC y EH Bild...   
4  Puigdemont: 'No sería ninguna tragedia una rep...   

                                       Titulo_Limpio  
0  the guardian va sanchez europa necesita apuest...  
1  revelan gobierno negocio liberacion mireles ca...  
2  ahora nunca joan fuster estatuto valenciano cu...  
3  iglesias alienta yolanda diaz erc eh bildu neg...  
4  puigdemont seria ninguna tragedia repeticion e...  
                                         Descripcion  \
0  El diario británico publicó este pasado jueves...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El valencianismo convoca en Castelló su fiesta...   
3  En política, igual que hay que negociar con lo...   
4  En u

### 3.2. Tokenización

Este código realiza la tokenización de texto. Esta tokenización se aplica a las columnas ya limpias del DataFrame (Titulo_Limpio y Descripcion_Limpia), generando nuevas columnas Titulo_Tokens y Descripcion_Tokens, donde cada texto se representa como una lista de palabras. Este paso es crucial porque permite a los modelos de análisis de texto procesar los datos de manera estructurada.


In [8]:
def tokenize_nltk(text):
    return word_tokenize(text, language="spanish")

# Aplicar tokenización en las columnas ya limpias
news_df['Titulo_Tokens'] = news_df['Titulo_Limpio'].astype(str).apply(tokenize_nltk)
news_df['Descripcion_Tokens'] = news_df['Descripcion_Limpia'].astype(str).apply(tokenize_nltk)

# Mostrar ejemplos
print("Comparación después de la tokenización:")
print(news_df[['Titulo_Limpio', 'Titulo_Tokens']].head())
print(news_df[['Descripcion_Limpia', 'Descripcion_Tokens']].head())



Comparación después de la tokenización:
                                       Titulo_Limpio  \
0  the guardian va sanchez europa necesita apuest...   
1  revelan gobierno negocio liberacion mireles ca...   
2  ahora nunca joan fuster estatuto valenciano cu...   
3  iglesias alienta yolanda diaz erc eh bildu neg...   
4  puigdemont seria ninguna tragedia repeticion e...   

                                       Titulo_Tokens  
0  [the, guardian, va, sanchez, europa, necesita,...  
1  [revelan, gobierno, negocio, liberacion, mirel...  
2  [ahora, nunca, joan, fuster, estatuto, valenci...  
3  [iglesias, alienta, yolanda, diaz, erc, eh, bi...  
4  [puigdemont, seria, ninguna, tragedia, repetic...  
                                  Descripcion_Limpia  \
0  diario britanico publico pasado jueves editori...   
1  revelan gobierno negocio liberacion mireles ca...   
2  valencianismo convoca castello fiesta grande c...   
3  politica igual negociar empresarios negociar g...   
4  entrevista

### 3.3. Normalización

Este código realiza la normalización del texto aplicando técnicas de stemming y lemmatización. La normalización se aplica a las columnas del DataFrame (Titulo_Tokens y Descripcion_Tokens), generando nuevas columnas Titulo_Normalizado y Descripcion_Normalizada, donde cada palabra es reducida a su raíz. Este paso es fundamental porque mejora la consistencia del texto y facilita el procesamiento de datos en modelos de análisis de texto.

In [10]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

# Descargar recursos necesarios para lematización
nltk.download('wordnet')
nltk.download('omw-1.4')

# Crear los objetos para stemming y lematización
stemmer = SnowballStemmer("spanish")
lemmatizer = WordNetLemmatizer()

def stem_words(words):
    """Aplica stemming para eliminar prefijos y sufijos en las palabras"""
    return [stemmer.stem(word) for word in words]

def lemmatize_verbs(words):
    """Aplica lematización para obtener la raíz de los verbos"""
    return [lemmatizer.lemmatize(word, wordnet.VERB) for word in words]

def stem_and_lemmatize(words):
    """Combina stemming y lematización para normalizar los datos"""
    stemmed = stem_words(words)
    lemmatized = lemmatize_verbs(words)
    return list(set(stemmed + lemmatized))  # Se usa set() para evitar duplicados

# Aplicar normalización a las columnas tokenizadas
news_df['Titulo_Normalizado'] = news_df['Titulo_Tokens'].apply(stem_and_lemmatize)
news_df['Descripcion_Normalizada'] = news_df['Descripcion_Tokens'].apply(stem_and_lemmatize)

# Mostrar ejemplos
print("Comparación después de la normalización:")
print(news_df[['Titulo_Tokens', 'Titulo_Normalizado']].head())
print(news_df[['Descripcion_Tokens', 'Descripcion_Normalizada']].head())


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Comparación después de la normalización:
                                       Titulo_Tokens  \
0  [the, guardian, va, sanchez, europa, necesita,...   
1  [revelan, gobierno, negocio, liberacion, mirel...   
2  [ahora, nunca, joan, fuster, estatuto, valenci...   
3  [iglesias, alienta, yolanda, diaz, erc, eh, bi...   
4  [puigdemont, seria, ninguna, tragedia, repetic...   

                                  Titulo_Normalizado  
0  [guardian, sanchez, va, guardi, apuest, europ,...  
1  [cambio, negoci, javier, duart, gobiern, javi,...  
2  [fuster, anos, valenciano, cumple, nunc, ahor,...  
3  [negoci, iglesias, investidura, alient, bloqu,...  
4  [eleccion, elecciones, ninguna, ningun, seri, ...  
                                  Descripcion_Tokens  \
0  [diario, britanico, publico, pasado, jueves, e...   
1  [revelan, gobierno, negocio, liberacion, mirel...   
2  [valencianismo, convoca, castello, fiesta, gra...   
3  [politica, igual, negociar, empresarios, negoc...   
4  [entrevis